In [1]:
# import dependencies
import io
import nltk
import json
import gzip
import torch
import string
import random
import operator
import jsonlines
import pandas as pd
import pickle as pkl
import numpy as np
from tqdm import tqdm
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import Dataset, RandomSampler, SequentialSampler, DataLoader
from sklearn.metrics import precision_recall_fscore_support, multilabel_confusion_matrix
random.seed(1)

In [2]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

FloatTensor = torch.cuda.FloatTensor if current_device == 'cuda' else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if current_device == 'cuda' else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if current_device == 'cuda' else torch.ByteTensor

In [3]:
def convCategories(categories, category_to_index):
    return [category_to_index[category] for category in categories]

In [4]:
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=1):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [5]:
def tokenize_dataset(dataset, word_to_index):
    _current_dictified = []
    for l in tqdm(dataset['tokens']):
        encoded_l = [word_to_index[i] if i in word_to_index else word_to_index['<UNK>'] for i in l]
        _current_dictified.append(encoded_l)
    return _current_dictified

In [6]:
class TensoredDataset(Dataset):
    def __init__(self, list_of_lists_of_tokens, labels):
        self.input_tensors = []
        self.target_tensors = []
        
        for i in range(0, len(list_of_lists_of_tokens)):
            self.input_tensors.append(torch.tensor(list_of_lists_of_tokens[i], dtype=torch.long))
            self.target_tensors.append(torch.tensor(labels[i], dtype=torch.long))
    
    def __len__(self):
        return len(self.input_tensors)
    
    def __getitem__(self, idx):
        # return a (input, target) tuple
        return (self.input_tensors[idx], self.target_tensors[idx])

In [7]:
def pad_list_of_tensors(list_of_tensors, pad_token):
    max_length = max([t.size(-1) for t in list_of_tensors])
    padded_list = []
    
    for t in list_of_tensors:
        #print(t.reshape(1, -1).shape)
        #print(torch.tensor([[pad_token]*(max_length - t.size(-1))])[0].shape)
        padded_tensor = torch.cat([t.reshape(1, -1), torch.tensor([[pad_token]*(max_length - t.size(-1))], dtype=torch.long)], dim = -1)
        padded_list.append(padded_tensor)
    padded_tensor = torch.cat(padded_list, dim=0)
    return padded_tensor

def transform_labels(target_list):
    padded_list = []
    for t in target_list:
        labels = t.unsqueeze(0)
        target = torch.zeros(labels.size(0), len(category_to_index)).scatter_(1, labels, 1)
        padded_list.append(target)
    padded_tensor = torch.cat(padded_list, dim=0)
    return padded_tensor

def pad_collate_fn(batch):
    # batch is a list of sample tuples
    input_list = [s[0] for s in batch]
    target_list = [s[1] for s in batch]
    
    #pad_token = persona_dict.get_id('<pad>')
    pad_token = word_to_index['<PAD>']
    
    input_tensor = pad_list_of_tensors(input_list, pad_token)
    #target_tensor = pad_list_of_tensors(target_list, pad_token)
    target_tensor = transform_labels(target_list)
    
    return input_tensor, target_tensor


In [8]:
class LSTModel(nn.Module):
    """
    This model combines embedding, rnn and projection layer into a single model
    """
    def __init__(self, options):
        super().__init__()
        
        # create each LM part here 
        self.lookup = nn.Embedding(num_embeddings=options['num_embeddings'], embedding_dim=options['embedding_dim'], padding_idx=options['padding_idx']).from_pretrained(weights_matrix)
        self.lstm = nn.LSTM(options['input_size'], options['hidden_size'], options['num_layers'], batch_first=True)
        self.projection = nn.Linear(options['hidden_size']*options['num_layers'], options['num_labels'])
        
    def forward(self, encoded_input_sequence):
        """
        Forward method process the input from token ids to logits
        """
        embeddings = self.lookup(encoded_input_sequence)
        lstm_outputs, (hn, cn) = self.lstm(embeddings)
        logits = self.projection(hn[-1])
        
        return logits

In [9]:
class StructuredSelfAttention(torch.nn.Module):
    """
    The class is an implementation of the paper A Structured Self-Attentive Sentence Embedding including regularization
    and without pruning. Slight modifications have been done for speedup
    """
   
    def __init__(self,batch_size,lstm_hid_dim,d_a,r,emb_dim=100,vocab_size=None,use_pretrained_embeddings = False,embeddings=None,n_classes = 1):
        """
        Initializes parameters suggested in paper
 
        Args:
            batch_size  : {int} batch_size used for training
            lstm_hid_dim: {int} hidden dimension for lstm
            d_a         : {int} hidden dimension for the dense layer
            r           : {int} attention-hops or attention heads
            max_len     : {int} number of lstm timesteps
            emb_dim     : {int} embeddings dimension
            vocab_size  : {int} size of the vocabulary
            use_pretrained_embeddings: {bool} use or train your own embeddings
            embeddings  : {torch.FloatTensor} loaded pretrained embeddings
            type        : [0,1] 0-->binary_classification 1-->multiclass classification
            n_classes   : {int} number of classes
 
        Returns:
            self
 
        Raises:
            Exception
        """
        super(StructuredSelfAttention,self).__init__()
       
        self.embeddings,emb_dim = self._load_embeddings(use_pretrained_embeddings,embeddings,vocab_size,emb_dim)
        self.lstm = torch.nn.LSTM(emb_dim,lstm_hid_dim,1,batch_first=True)
        self.linear_first = torch.nn.Linear(lstm_hid_dim,d_a)
        self.linear_first.bias.data.fill_(0)
        self.linear_second = torch.nn.Linear(d_a,r)
        self.linear_second.bias.data.fill_(0)
        self.n_classes = n_classes
        self.linear_final = torch.nn.Linear(lstm_hid_dim,self.n_classes)
        self.batch_size = batch_size       
        self.lstm_hid_dim = lstm_hid_dim
        self.hidden_state = self.init_hidden()
        self.r = r
                 
    def _load_embeddings(self,use_pretrained_embeddings,embeddings,vocab_size,emb_dim):
        """Load the embeddings based on flag"""
       
        if use_pretrained_embeddings is True and embeddings is None:
            raise Exception("Send a pretrained word embedding as an argument")
           
        if not use_pretrained_embeddings and vocab_size is None:
            raise Exception("Vocab size cannot be empty")
   
        if not use_pretrained_embeddings:
            word_embeddings = torch.nn.Embedding(vocab_size,emb_dim,padding_idx=0)
            
        elif use_pretrained_embeddings:
            word_embeddings = torch.nn.Embedding(embeddings.size(0), embeddings.size(1))
            word_embeddings.weight = torch.nn.Parameter(embeddings)
            emb_dim = embeddings.size(1)
            
        return word_embeddings,emb_dim
       
        
    def softmax(self,input, axis=1):
        """
        Softmax applied to axis=n
 
        Args:
           input: {Tensor,Variable} input on which softmax is to be applied
           axis : {int} axis on which softmax is to be applied
 
        Returns:
            softmaxed tensors
 
       
        """
 
        input_size = input.size()
        trans_input = input.transpose(axis, len(input_size)-1)
        trans_size = trans_input.size()
        input_2d = trans_input.contiguous().view(-1, trans_size[-1])
        soft_max_2d = F.softmax(input_2d)
        soft_max_nd = soft_max_2d.view(*trans_size)
        return soft_max_nd.transpose(axis, len(input_size)-1)
       
        
    def init_hidden(self):
        return (Variable(torch.zeros(1,self.batch_size,self.lstm_hid_dim)),Variable(torch.zeros(1,self.batch_size,self.lstm_hid_dim)))
       
        
    def forward(self,x):
        embeddings = self.embeddings(x)       
        outputs, self.hidden_state = self.lstm(embeddings)       
        x = F.tanh(self.linear_first(outputs))       
        x = self.linear_second(x)       
        x = self.softmax(x,1)       
        attention = x.transpose(1,2)       
        sentence_embeddings = attention@outputs       
        avg_sentence_embeddings = torch.sum(sentence_embeddings,1)/self.r
       
        output = self.linear_final(avg_sentence_embeddings)
        return output, attention
       
	   
	#Regularization
    def l2_matrix_norm(self,m):
        """
        Frobenius norm calculation
 
        Args:
           m: {Variable} ||AAT - I||
 
        Returns:
            regularized value
 
       
        """
        return torch.sum(torch.sum(torch.sum(m**2,1),1)**0.5).type(torch.DoubleTensor)

In [10]:
OUTPUT_FILE = '/scratch/sa5154/Capstone/Models/wikitext_tokenized.p'
wiki_df =  pkl.load(open(OUTPUT_FILE, "rb"))

In [11]:
categories = []
for i in list(wiki_df['mid_level_categories']):
    categories.extend(i)
categories = sorted(list(set(categories)))

In [12]:
category_to_index = {categories[i]:i for i in range(0, len(categories))}
index_to_category = {v:k for k, v in category_to_index.items()}

In [13]:
wiki_df['category_tokens'] = wiki_df.apply(lambda row: convCategories(row['mid_level_categories'], category_to_index), axis=1)

In [14]:
wiki_df = wiki_df[wiki_df.astype(str)['category_tokens'] != '[]']
wiki_df = wiki_df[wiki_df.astype(str)['tokens'] != '[]']
wiki_df = wiki_df.reset_index().drop('index', axis = 1)

In [15]:
wiki_train, wiki_valid, wiki_test = train_validate_test_split(wiki_df)

In [16]:
y_train = list(wiki_train['category_tokens'])
y_val = list(wiki_valid['category_tokens'])

In [17]:
vocab = sorted(set([y for x in list(wiki_train['tokens']) for y in x]))

In [18]:
len(vocab)

595516

In [19]:
word_to_index = {"<PAD>":0, "<UNK>":1}
for word in vocab:
    if(word not in word_to_index):
        word_to_index[word]=len(word_to_index)
index_to_word = {v:k for k, v in word_to_index.items()}

In [20]:
wiki_tokenized_train = tokenize_dataset(wiki_train, word_to_index)

100%|██████████| 79968/79968 [00:08<00:00, 8942.88it/s] 


In [21]:
wiki_tokenized_val = tokenize_dataset(wiki_valid, word_to_index)

100%|██████████| 9996/9996 [00:01<00:00, 9028.84it/s]


In [22]:
wiki_tokenized_datasets = {}
wiki_tokenized_datasets['train'] = wiki_tokenized_train
wiki_tokenized_datasets['val'] = wiki_tokenized_val

In [23]:
wiki_tensor_dataset = {}
wiki_tensor_dataset['train'] = TensoredDataset(wiki_tokenized_datasets['train'], y_train)
wiki_tensor_dataset['val'] = TensoredDataset(wiki_tokenized_datasets['val'], y_val)

In [24]:
wiki_loaders = {}

batch_size = 32

for split, wiki_dataset in wiki_tensor_dataset.items():
    wiki_loaders[split] = DataLoader(wiki_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate_fn)

In [25]:
def Validate(model):
    valid_loss_cache = 0
    all_targets = []
    all_logits = []
    model.eval()
    with torch.no_grad():
        for i, (inp, target) in enumerate(wiki_loaders['val']):
            inp = inp.to(current_device)
            target = target.to(current_device)
            logits, attention = model(inp)
            loss = criterion(logits, target)
            m = nn.Sigmoid()
            logits = m(logits)
            logits = logits.cpu().detach().numpy()
            target = target.cpu().numpy()
            all_targets.append(target)
            all_logits.append(logits)
            valid_loss_cache += loss.item()

        avg_val_loss = valid_loss_cache / (i+1)
        all_logits = np.concatenate(all_logits, axis=0)
        all_targets = np.concatenate(all_targets, axis=0)
        all_logits[all_logits > 0.5] = 1
        all_logits[all_logits <= 0.5] = 0
        prec_macro, rec_macro, f_score_macro, _ = precision_recall_fscore_support(all_targets, all_logits, average = 'macro')
        print('Validation macro prec: {}, rec:{}, f_score:{}'.format(prec_macro, rec_macro, f_score_macro))
        prec_micro, rec_micro, f_score_micro, _ = precision_recall_fscore_support(all_targets, all_logits, average = 'micro')
        print('Validation micro prec: {}, rec:{}, f_score:{}'.format(prec_micro, rec_micro, f_score_micro))
        print('Validation loss = {:.{prec}f}'.format(avg_val_loss, prec=4))
        return f_score_micro

# Pre Trained Embeddings

In [26]:
#Loading pre trained fastText word embeddings
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    data = {}
    for line in tqdm(fin):
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = list(map(float, tokens[1:]))
    return data

fasttext_emb = load_vectors("wiki.en.align.vec")

2519371it [02:57, 14196.83it/s]


In [27]:
#Creating the weight matrix for pretrained word embeddings
vocab_size = len(index_to_word)
embed_dim = len(fasttext_emb["apple"])
weights_matrix = np.zeros((vocab_size,embed_dim))

words_found = 0
for i, word in enumerate(word_to_index):
    try: 
        weights_matrix[i] = fasttext_emb[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(embed_dim))
weights_matrix = torch.FloatTensor(weights_matrix)

In [28]:
print("Total words in vocab: {}".format(len(vocab)))
print("No. of words from vocab found in fastText: {}".format(words_found))

Total words in vocab: 595516
No. of words from vocab found in fastText: 470498


In [29]:
options = {
    'num_embeddings': len(word_to_index),
    'embedding_dim': weights_matrix.size(1),
    'num_labels':44,
    'padding_idx': word_to_index['<PAD>'],
    'input_size': weights_matrix.size(1),
    'hidden_size': 64,
    'num_layers': 1,
}

#model = LSTModel(options).to(current_device)
model = StructuredSelfAttention(batch_size,64,64,1,emb_dim=100,vocab_size=len(vocab),use_pretrained_embeddings = True,embeddings=weights_matrix,n_classes = 44).to(current_device)


criterion = nn.BCEWithLogitsLoss()

model_parameters = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(model_parameters, lr=0.001)


In [30]:
model

StructuredSelfAttention(
  (embeddings): Embedding(595518, 300)
  (lstm): LSTM(300, 64, batch_first=True)
  (linear_first): Linear(in_features=64, out_features=64, bias=True)
  (linear_second): Linear(in_features=64, out_features=1, bias=True)
  (linear_final): Linear(in_features=64, out_features=44, bias=True)
)

In [31]:
best_f_score_micro = -1
count = 0
PATH = '/scratch/sa5154/Capstone/Models/LSTM_Attention_Pretrained_epoch_30.pth'
for epoch_number in range(10):
    print("Running Epoch:{}".format(epoch_number + 1))
    avg_loss = -1
    # do train
    model.train()

    train_loss_cache = 0

    for i, (inp, target) in enumerate(wiki_loaders['train']):
        model.train()
        optimizer.zero_grad()
        inp = inp.to(current_device)
        target = target.to(current_device)
        logits, _ = model(inp)
        loss = criterion(logits, target)
        train_loss_cache += loss.item()
       
        loss.backward()
        optimizer.step()

        if i % 500 == 0:
            avg_loss = train_loss_cache/(i+1)
            print('Step {} avg train loss = {:.{prec}f}'.format(i, avg_loss, prec=4))

            #do valid
            f_score_micro = Validate(model)
            if( f_score_micro > best_f_score_micro):
                best_f_score_micro = f_score_micro
                torch.save({
                        'state_dict': model.state_dict()
                        }, PATH)
                print("Model saved!")

Running Epoch:1
Step 0 avg train loss = 0.7146


/home/sa5154/miniconda3/lib/python3.7/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/sa5154/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/sa5154/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Validation macro prec: 0.03392667475920414, rec:0.5102942067374169, f_score:0.04402346269135621
Validation micro prec: 0.04092312821322665, rec:0.5451411207853678, f_score:0.07613117510007059
Validation loss = 0.6868
Model saved!
Step 500 avg train loss = 0.1488
Validation macro prec: 0.0, rec:0.0, f_score:0.0
Validation micro prec: 0.0, rec:0.0, f_score:0.0
Validation loss = 0.1237


/home/sa5154/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Step 1000 avg train loss = 0.1354
Validation macro prec: 0.014155199639070607, rec:0.021994942540246865, f_score:0.017224983578914346
Validation micro prec: 0.6228287841191067, rec:0.20534096885408754, f_score:0.3088551966600747
Validation loss = 0.1146
Model saved!
Step 1500 avg train loss = 0.1228
Validation macro prec: 0.16849045557460834, rec:0.07836493356598594, f_score:0.09176697091052408
Validation micro prec: 0.9104685110524121, rec:0.38269152106585635, f_score:0.5388792890644286
Validation loss = 0.0847
Model saved!
Step 2000 avg train loss = 0.1106
Validation macro prec: 0.28754929927613887, rec:0.15595779888146685, f_score:0.1780293489536967
Validation micro prec: 0.8795327102803738, rec:0.5499327996260153, f_score:0.6767339014130083
Validation loss = 0.0669
Model saved!
Running Epoch:2
Step 0 avg train loss = 0.0640
Validation macro prec: 0.3520114203631843, rec:0.2114803211689487, f_score:0.24348594589537906
Validation micro prec: 0.8770617105157293, rec:0.6370011102670484

In [31]:
PATH = '/scratch/sa5154/Capstone/Models/LSTM_Attention_Pretrained_epoch_30.pth'
model = StructuredSelfAttention(batch_size,64,64,1,emb_dim=100,vocab_size=len(vocab),use_pretrained_embeddings = True,embeddings=weights_matrix,n_classes = 44).to(current_device)
model.load_state_dict(torch.load(PATH)['state_dict'])
model.eval()

StructuredSelfAttention(
  (embeddings): Embedding(595518, 300)
  (lstm): LSTM(300, 64, batch_first=True)
  (linear_first): Linear(in_features=64, out_features=64, bias=True)
  (linear_second): Linear(in_features=64, out_features=1, bias=True)
  (linear_final): Linear(in_features=64, out_features=44, bias=True)
)

In [36]:
Validate(model)

/home/sa5154/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Validation macro prec: 0.6733391704836106, rec:0.49977193784877055, f_score:0.5531555197979134
Validation micro prec: 0.8611327617563004, rec:0.7747326593817565, f_score:0.8156510504783292
Validation loss = 0.0433


/home/sa5154/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.8156510504783292

In [37]:
def printText(num):
    in_ = inp[num].cpu().detach().numpy()
    str_ = ''
    for i in in_:
        if(i!=0):
            str_ += index_to_word[i]
            str_+=' '
    return str_

In [38]:
def showAtttention(num):
    att = attention[num].cpu().detach().numpy()
    in_ = inp[num].cpu().detach().numpy()
    targ = target[num].cpu().detach().numpy()
    pred = logits[num]
    att_dict = {}
    for i in range(0, len(att)):
        if(in_[i]!=0):
            if index_to_word[in_[i]] not in att_dict:
                att_dict[index_to_word[in_[i]]] = [att[i]]
            else:
                att_dict[index_to_word[in_[i]]].append(att[i])
    for key in att_dict.keys():
        att_dict[key] = np.mean(att_dict[key])
        
    att_dict = sorted(att_dict.items(), key=operator.itemgetter(1), reverse = True)
    label = [index_to_category[i] for i in range(len(targ)) if targ[i] == 1] 
    pred =  [index_to_category[i] for i in range(len(pred)) if pred[i] == 1] 
    str_ = printText(num)
    return label, pred, att_dict, str_

In [39]:
all_targets = []
all_logits = []
for i, (inp, target) in enumerate(wiki_loaders['val']):
    inp = inp.to(current_device)
    logits, attention = model(inp)
    m = nn.Sigmoid()
    logits = m(logits)
    logits = logits.cpu().detach().numpy()
    logits[logits > 0.5] = 1
    logits[logits <= 0.5] = 0
    attention.squeeze_()
    break
#     target = target.detach().numpy()
#     all_targets.append(target)
#     all_logits.append(logits)

/home/sa5154/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


## Comparison to TF-IDF

In [40]:
train_list =[]
for index, row in wiki_train.iterrows():
    train_list.append(' '.join(row.tokens))

val_list =[]
for index, row in wiki_valid.iterrows():
    val_list.append(' '.join(row.tokens))


In [41]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_train_scores = tfidf.fit_transform(train_list)

In [42]:
def show_TFIDF_Attention(num):
    label, pred, att_dict, text = showAtttention(num)
    tfidf_scores = tfidf.transform([text])
    terms = tfidf.get_feature_names()
    data = []
    for col, term in enumerate(terms):
        data.append( (term, tfidf_scores[0,col] ))

    ranking = pd.DataFrame(data, columns=['term','rank'])
    ranking = ranking.sort_values('rank', ascending=False)
    top_arr_words = set([w for w,s in att_dict[:10]])
    top_tfidf_words = set(ranking[:10]['term'])
    common_len = len(top_arr_words.intersection(top_tfidf_words))
    print('The number of words common in top 10 words are: {}'.format(common_len))
    return label, pred, att_dict, ranking

In [43]:
num = random.randint(0,32)
label, pred, att_dict, tfidf_ranking = show_TFIDF_Attention(num)

The number of words common in top 10 words are: 5


In [1]:
tfidf_ranking[:10]

In [45]:
att_dict[:10]

[('elections', 0.21944173),
 ('senate', 0.0134309335),
 ('states', 0.0038709235),
 ('labor', 0.0015933439),
 ('attorney', 0.0007289613),
 ('new', 0.00069535465),
 ('election', 0.0004225585),
 ('case', 0.00032382517),
 ('united', 0.0002881809),
 ('clifford', 0.00026884358)]

In [51]:
label

['Geography.Americas']

In [53]:
pred

['Geography.Americas', 'History_And_Society.Politics and government']

In [46]:
def create_per_class_tables(loader, model, device, class_names, threshold=0.5):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    model.eval()
    outputs_list_nc = []
    true_list_nc = []
    with torch.no_grad():
        for data, labels in loader:
            data_batch, label_batch = data.to(device), labels.float()
            logits, _ = model(data_batch)
            outputs_bc = torch.sigmoid(logits)
            outputs_bc = outputs_bc.detach().cpu().numpy().astype(np.float)
            outputs_bc = (outputs_bc > threshold)
            outputs_list_nc.append(outputs_bc)
            true_list_nc.append(label_batch.detach().cpu().numpy().astype(np.float))
    # to np.array
    outputs_list_nc = np.vstack(outputs_list_nc)
    true_list_nc = np.vstack(true_list_nc)
    
    # per class counts
    counts_c = true_list_nc.sum(axis=0)
    
    # per class confusion matrix: TN, FN, TP, FP
    confusion_matrix_c22 = multilabel_confusion_matrix(
        true_list_nc,
        outputs_list_nc,
    )
    confusion_matrix_c4 = confusion_matrix_c22.reshape(-1, 4)
    
    # per class precision, recall, f-score
    precision_c, recall_c, f1_c, _ = precision_recall_fscore_support(
        true_list_nc,
        outputs_list_nc,
        average=None
    )
    
    # combine all metrics in a dict
    per_class_metrics = {
        "class_name": class_names,
        "count": counts_c,
        "TN": confusion_matrix_c4[:,0], 
        "FN": confusion_matrix_c4[:,2],
        "TP": confusion_matrix_c4[:,3],
        "FP": confusion_matrix_c4[:,1],
        "precision": precision_c, 
        "recall": recall_c, 
        "f1": f1_c
    }
    return pd.DataFrame(per_class_metrics)

In [47]:
per_class_metrics = create_per_class_tables(wiki_loaders['val'], model, current_device, categories, threshold=0.5)

/home/sa5154/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [50]:
per_class_metrics.sort_values('f1', ascending=False)

,class_name,count,TN,FN,TP,FP,precision,recall,f1
7,Culture.Language and literature,3631.0,6226,196,3435,139,0.961108,0.946020,0.953505
17,Geography.Antarctica,27.0,9967,1,26,2,0.928571,0.962963,0.945455
13,Culture.Sports,1567.0,8299,151,1416,130,0.915912,0.903638,0.909733
31,STEM.Biology,771.0,9175,99,672,50,0.930748,0.871595,0.900201
23,Geography.Oceania,468.0,9463,52,416,65,0.864865,0.888889,0.876712
41,STEM.Space,67.0,9927,14,53,2,0.963636,0.791045,0.868852
30,History_And_Society.Transportation,551.0,9380,80,471,65,0.878731,0.854809,0.866605
18,Geography.Asia,1399.0,8449,338,1061,148,0.877585,0.758399,0.813650
9,Culture.Music,435.0,9501,96,339,60,0.849624,0.779310,0.812950
5,Culture.Games and toys,109.0,9881,33,76,6,0.926829,0.697248,0.795812
